# Haplotype analysis *Coeaexg*

### Color palettes

In [1]:
# define dictionary of pop colors
pop_colours = {
 'Moshi' : "lightgreen",
 'Muleba' : "darkgreen",
}

dup_colours = {
    'Dup present' : 'mediumorchid',
    'Dup absent' : 'lightgray',
    '4 extra copies': 'purple',
    '3 extra copies': 'mediumorchid',
    '2 extra copies': 'orange',
    '1 extra copy': 'pink',
    'No CNV': 'lightgray'
}


### Import packages

In [2]:
import malariagen_data
import allel
import numpy as np
import pandas as pd
import re
import scipy.cluster
from scipy.stats import spearmanr
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns



### Load CNV data

In [3]:
# Identify samples that have Cyp6_Dup31
disc_cnv_fn = '../Ag1000G_CNV_data/v3.7_1246-VO-TZ-KABULA-VMF00185/target_regions_analysis/focal_region_CNV_table.csv'
discordant_read_calls = pd.read_csv(disc_cnv_fn, sep = '\t', index_col = 0)
has_Dup = discordant_read_calls['Coeaexf_Dup2'].values
Dup_calls = has_Dup.astype('int')

# In order to known the copy number of each Dup specifically, we need to ignore samples that also
# have another CNV. 
other_coeaexf_cnvs = (
    discordant_read_calls.columns.str.contains('Coeaexf') &
    ~discordant_read_calls.columns.str.contains('Dup0') &
    ~discordant_read_calls.columns.str.contains('Dup2') 
)
other_coeaexf_calls = discordant_read_calls.loc[:, other_coeaexf_cnvs]
has_other_coeaexf_cnv = np.sum(other_coeaexf_calls, 1).values > 0

# Now we can exclude samples that have both the focal and another CNV allele, or that have the 
# allele and are of high coverage variance (because we won't be able to assign copy number correctly)
exclude_samples = (  
    has_Dup & (
        has_other_coeaexf_cnv | 
        discordant_read_calls["High.var.sample"]
    )                  
).values



FileNotFoundError: [Errno 2] No such file or directory: '../Ag1000G_CNV_data/v3.7_1246-VO-TZ-KABULA-VMF00185/target_regions_analysis/focal_region_CNV_table_extras.csv'

### Load metadata

In [ ]:
ag3 = malariagen_data.Ag3(pre=True)

sample_sets = ['1246-VO-TZ-KABULA-VMF00185']
meta = ag3.sample_metadata(sample_sets = sample_sets).set_index('sample_id')
meta['location'] = meta['location'].str.replace('_.*', '', regex=True)

# Make sure the samples are in the same order as the CNV data (they should be anyway, but 
# just in case)
meta = meta.loc[discordant_read_calls.index, :]

# Get haplotype-level metadata
hap_meta = pd.DataFrame(np.repeat(meta.values, 2, axis = 0))
hap_meta.columns = meta.columns
hap_meta.index = [np.repeat(meta.index, 2)[i] + ['a', 'b'][i%2] for i in range(len(hap_meta))]

hap_meta_Dup = hap_meta.loc[~np.repeat(exclude_samples, 2), :]

hap_meta


In [ ]:
gene_copy_number = pd.read_csv('../Ag1000G_CNV_data/v3.7_1246-VO-TZ-KABULA-VMF00185/modal_CNVs/modal_copy_number_arabiensis.csv', sep = '\t', index_col = 0)

# Define genes of interest
coeaexf_genes = dict(AGAP006227='Coeae1f',
                     AGAP006228='Coeae2f')

# Select data for genes of interest, excluding samples with poor quality HMM data
coeae2f_gene_copy_number = (
    gene_copy_number.loc[:, coeaexf_genes.keys()]
    .rename(coeaexf_genes, axis='columns')
)['Coeae2f'].values

Dup_copy_number = coeae2f_gene_copy_number[~exclude_samples] * Dup_calls[~exclude_samples]

display(pd.crosstab(meta.loc[~exclude_samples, 'location'], Dup_copy_number))


The copy number can get above 2, so probably isn't a simple duplication

### Load the haplotype data

In [ ]:
region='2L:28300000-28800000'

haps = (
    ag3.haplotypes(region = region, 
                   analysis = 'arab', 
                   sample_sets = sample_sets)
    .set_index(samples = 'sample_id')
    .sel(samples = discordant_read_calls.index)
)

haps_Dup = haps.isel(samples = ~exclude_samples)


In [ ]:
# Create haplotype array
hap_array_Dup = allel.GenotypeArray(haps_Dup.call_genotype).to_haplotypes()

# Get allele counts
ac = hap_array_Dup.count_alleles()

# Identify segregating non-singletons. 
non_singleton = ac.min(1) > 1

# Filter the datasets, removing non-segregating and singleton variants 
haps_Dup = haps_Dup.isel(variants = np.where(non_singleton)[0])
hap_array_Dup = hap_array_Dup[non_singleton]


In [ ]:
# Get the SNP genotypes coded as 0, 1, 2 (wt, het, homozygote mutant). Since all phased SNPs
# are biallelic, this is a suitable way to code genotype. 
genotypes = haps_Dup.call_genotype.values.sum(2)
genotypes

### Identify proxy SNPs

In [ ]:
# We look for SNPs whose presence / absence patterns are perfectly correlated with Dup31. 
SNP_cor = np.apply_along_axis(lambda x: spearmanr((x>0).astype('int'), 
                                                  Dup_calls[~exclude_samples]), 
                                    1, 
                                    genotypes)[:, 0]
SNP_cor = np.abs(SNP_cor)
print(f'The maximum correlation coefficient with the Dup is {np.nanmax(SNP_cor)}')

In [ ]:
Dup_proxy = genotypes[np.nanargmax(SNP_cor)]

# Let's look at the association of zygosity at those SNPs and copy number. 

print('Association of Dup2 proxy zygosity and Dup2 copy number in Moshi:')
display(
    pd.crosstab(Dup_copy_number[(meta.loc[~exclude_samples, :].location == 'Moshi').values],
                Dup_proxy[(meta.loc[~exclude_samples, :].location == 'Moshi').values])
)

print('Association of Dup2 proxy zygosity and Dup2 copy number in Muleba:')
display(
    pd.crosstab(Dup_copy_number[(meta.loc[~exclude_samples, :].location == 'Muleba').values],
                Dup_proxy[(meta.loc[~exclude_samples, :].location == 'Muleba').values])
)



It's a half decent proxy, but produces both false positive and false negatives. 

In [ ]:
# Get the haplotypes rather than genotypes for the Dup2 proxy
Dup_hap_proxy = hap_array_Dup[np.nanargmax(SNP_cor)]
Dup_hap_proxy

### Haplotype plotting functions

In [ ]:
def find_clusters(dist, n, threshold=0.001, method='complete'):
        # build hierarchy
        clust = scipy.cluster.hierarchy.linkage(dist, method=method)
        # find clusters
        f = scipy.cluster.hierarchy.fcluster(clust, threshold,
                                             criterion='distance')
        # compute cluster sizes
        fsz = np.bincount(f)
        # sort largest first
        fsort = np.argsort(fsz)[::-1]
        # take largest n
        fsort = fsort[:n]
        # get haplotype indices for each cluster
        clusters = [set(np.nonzero(f == i)[0]) for i in fsort]
        return clusters

# Some plotting functions
def truspan(cluster, r):
    # get the index of the cluster haps in the dendrogram list of all haps
    cluster_leaves = sorted([r['leaves'].index(i) for i in cluster])
    # are these indices monotonic - they should be!
    x = np.asarray(cluster_leaves)
    dx = np.diff(x)
    mon = np.all(dx == 1)
    assert mon
    return min(cluster_leaves), max(cluster_leaves)

def plot_dendrogram(zhier, ax, h, method='complete', color_threshold=0, above_threshold_color='k'):

    # plot dendrogram
    sns.despine(ax=ax, offset=5, bottom=True, top=False)
    r = scipy.cluster.hierarchy.dendrogram(zhier, no_labels=True, count_sort=True,
                                           color_threshold=color_threshold,
                                           above_threshold_color=above_threshold_color,
                                           ax=ax)
    xmin, xmax = ax.xaxis.get_data_interval()
    xticklabels = np.array(list(range(0, h.shape[1], 200)) + [h.shape[1]])
    xticks = xticklabels / h.shape[1]
    xticks = (xticks * (xmax - xmin)) + xmin
    ax.set_xticks(xticks)
    ax.set_xticklabels(xticklabels)
    #ax.set_xlabel('Haplotypes')
    ax.xaxis.set_label_position('top')
    ax.set_ylim(bottom=-.0001)

    ax.set_ylabel(r'$d_{xy}$')
    ax.autoscale(axis='x', tight=True)
    return r

def draw_hap_cluster_plot(z, r, h, 
                          pop_colour_scheme, cnv_colour_scheme, 
                          labels_for_pop_colours, labels_for_cnv_colours, 
                          cnv_name,
                          add_legend = True, title = '',
                          fn = None):

    gs = GridSpec(3, 1, height_ratios=[6.0, 0.5, 0.5])
    fig = plt.figure(figsize=(15, 5))

    plt.subplots_adjust(hspace=0.02)
    ax1 = plt.subplot(gs[0])
    sns.despine(ax=ax1, offset=5, bottom=True, top=True)
    _ = plot_dendrogram(z, ax1, h)
    ax1.spines['top'].set_visible(False)
    ax1.set_xticks([])
    ax1.set_ylim(0)

    ax_pops = fig.add_subplot(gs[1])

    x_pop = labels_for_pop_colours.take(r['leaves'])
    hap_pop_clrs = [pop_colour_scheme[p] for p in x_pop]
    ax_pops.broken_barh(xranges=[(i, 1) for i in range(h.shape[1])],
                        yrange=(0, 1),
                        color=hap_pop_clrs);
    sns.despine(ax=ax_pops, offset=5, left=True, bottom=True)
    
    ax_pops.set_xticks([])
    ax_pops.set_yticks([])
    ax_pops.set_xlim(0, h.shape[1])
    ax_pops.yaxis.set_label_position('left')
    ax_pops.set_ylabel('Population', rotation=0, ha='right', va='center')
    
    ax_cnv = fig.add_subplot(gs[2])

    x_cnv = labels_for_cnv_colours.take(r['leaves'])
    hap_cnv_clrs = [cnv_colour_scheme[p] for p in x_cnv]
    ax_cnv.broken_barh(xranges=[(i, 1) for i in range(h.shape[1])],
                       yrange=(0, 1),
                       color=hap_cnv_clrs);
    sns.despine(ax=ax_cnv, offset=5, left=True, bottom=True, top=True)

    ax_cnv.set_xticks([])
    ax_cnv.set_yticks([])
    ax_cnv.set_xlim(0, h.shape[1])
    ax_cnv.yaxis.set_label_position('left')
    ax_cnv.set_ylabel(cnv_name, rotation=0, ha='right', va='center')

    if add_legend:
        plot_x_range = ax1.get_xlim()[1] - ax1.get_xlim()[0]
        plot_y_range = ax1.get_ylim()[1] - ax1.get_ylim()[0]
        legend_x = ax1.get_xlim()[0] + plot_x_range * 0.95
            
        unique_label_cnv = np.unique(x_cnv)
        for i, k in enumerate(unique_label_cnv[::-1]):
            legend_y = ax1.get_ylim()[0] + plot_y_range * (0.2 + 0.1 * i)
            ax1.add_patch(mpl.patches.Rectangle((legend_x,legend_y), plot_x_range/50,plot_y_range/15, color = cnv_colour_scheme[k]))
            ax1.text(legend_x*1.03,legend_y + plot_y_range / 50, k)

        
        unique_label_pop = np.unique(x_pop)
        for j, k in enumerate(unique_label_pop[::-1]):
            legend_y = ax1.get_ylim()[0] + plot_y_range * (0.2 + 0.1 * (i + 2 + j))
            ax1.add_patch(mpl.patches.Rectangle((legend_x,legend_y), plot_x_range/50,plot_y_range/15, color = pop_colour_scheme[k]))
            ax1.text(legend_x*1.03,legend_y + plot_y_range / 50, k)



    ax1.set_title(f'{title}')
    ax1.autoscale()
    fig.patch.set_alpha(1)
    
    if fn != None:
        ext = re.sub('.*\.', '', fn)
        plt.savefig(fn, format = ext)




### Haplotype clustering

In [ ]:
# Home in on the 500 SNPs around the region of interest
focal_pos = 28548072 # Middle of the two genes Coeae1f and Coeae2f
window_size = 500  # Measured in SNPs


In [ ]:
nearest_snp_index = np.argmin(np.abs(haps_Dup.variant_position - focal_pos).values)
haptree_region_indices = range(nearest_snp_index - window_size//2, nearest_snp_index + window_size//2)
haptree_region = '2L:' + '-'.join(haps_Dup.isel(variants = haptree_region_indices).variant_position.values[[0,-1]].astype('str'))
loc_hap = hap_array_Dup[haptree_region_indices]
# loc_hap is the haplotype array for just those 500 SNPs. 

# Calculate the distance matrix 
dist = allel.pairwise_distance(loc_hap, metric = 'hamming')
site_filter = ag3.snp_calls(region=haptree_region, sample_sets="3.7")['variant_filter_pass_arab']
n_bases = np.sum(site_filter.values)
dist_dxy = dist * loc_hap.n_variants / n_bases

z = scipy.cluster.hierarchy.linkage(dist_dxy, method="complete")
r = scipy.cluster.hierarchy.dendrogram(
        z, no_labels=True, count_sort=True,
        color_threshold=0, no_plot=True,
        above_threshold_color='k')

# Now let's plot the haplotype clustering tree
Dup_hap_status = np.array(['Dup absent', 'Dup present'])[Dup_hap_proxy]
draw_hap_cluster_plot(z, r, loc_hap, 
                      pop_colour_scheme = pop_colours, 
                      cnv_colour_scheme = dup_colours,
                      labels_for_pop_colours = hap_meta_Dup['location'], 
                      labels_for_cnv_colours = Dup_hap_status, 
                      cnv_name = 'Coeaexf_Dup2',
                      title = 'Haplotypes around Coeaexf',
                      fn = 'Coeaexf_haplotype_clustering.png'
)

Given that we don't have a SNP with perfect correlation, let's try just looking at copy number of the SNP on the genotype

In [ ]:
#Dup_status = np.array(['Dup absent', 'Dup present'])[discordant_read_calls['Coeaexf_Dup2'].astype('int')]
Dup_status = np.array(['No CNV', '1 extra copy', '2 extra copies', '3 extra copies', '4 extra copies'])[np.repeat(Dup_copy_number, 2)]

draw_hap_cluster_plot(z, r, loc_hap, 
                      pop_colour_scheme = pop_colours, 
                      cnv_colour_scheme = dup_colours,
                      labels_for_pop_colours = hap_meta['location'], 
                      labels_for_cnv_colours = Dup_status, 
                      cnv_name = 'Coeaexf_Dup2',
                      title = 'Haplotypes around Coeaexg',
                      fn = 'Coeaexf_haplotype_clustering_copy_number.png'
)

So there are still members of the swept cluster that don't have the haplotype, which suggests that the CNV appeared on the existing swept cluster. 

In [ ]:
# Can we find correlated SNPs that aren't among the phased haplotypes
snps = (
    ag3.snp_calls(region = region, 
                   sample_sets = sample_sets)
    .set_index(samples = 'sample_id')
    .sel(samples = discordant_read_calls.index)
)
snps

In [ ]:
# That's a lot bigger than the haplotypes. 
# Let's start by removing SNPs that are non-segregating or singletons in this dataset
gen_array = allel.GenotypeArray(snps.call_genotype)
gen_ac = gen_array.count_alleles()
gen_ac

In [ ]:
gen_is_segregating = gen_ac.is_segregating()
gen_segregating_ac = gen_ac[gen_is_segregating, :]
gen_segregating_ac

In [ ]:
min_maf = 50
keep_snp = np.max(gen_segregating_ac, 1) <= (gen_array.shape[1]*2 - min_maf)
filtered_snps = snps.sel(variants = gen_is_segregating).sel(variants = keep_snp)
filtered_snps

In [ ]:
# We're going to need a separate test for every allele, since you can't represent a genotype
# with more than two alleles as a single number.
def all_alleles_spearman(gen, thresh_allele_count = 100):
    output = [0]*4
    for i in range(4):
        allele_gen = gen == i
        if np.sum(allele_gen) < thresh_allele_count:
            p = np.nan
        else:
            p = spearmanr(np.any(allele_gen, 1).astype('int'), Dup_calls)[0]
        output[i] = p
    return(np.array(output))


In [ ]:
all_genotypes = filtered_snps.call_genotype.values


In [ ]:
gen_SNP_cor = np.array([all_alleles_spearman(all_genotypes[i, :, :]) for i in range(all_genotypes.shape[0])])
gen_SNP_cor = np.nanmax(np.abs(gen_SNP_cor), 1)
print(f'The maximum correlation coefficient with the Dup is {np.nanmax(gen_SNP_cor)}')

That's not much better than when looking at phased haplotypes. 